In [2]:
import base64
from PIL import Image, ImageOps,ImageDraw,ImageFont
import qrcode
import tempfile
from pathlib import Path
import io
from zipfile import ZipFile

In [3]:
def overlay_images(background_path, qr_code_img, position, qr_code_size):
    # Open the background image
    background = Image.open(background_path).convert("RGBA").copy()
    
    # Open the overlay image
    qr_code = base64.b64decode(qr_code_img)
    
    buf = io.BytesIO(qr_code)
    
    qr_code = Image.open(buf)
    
    # Resize the overlay image if needed (optional)
    qr_code = qr_code.resize(qr_code_size)

    # Overlay the image
    background.paste(qr_code, position, qr_code)

    return background

# Cantina Volpi Comanda

- Background Info:
    - Height: 1772px
    - Widht:  1181px

- Qr Code Info:
    - Position(320, 787)
    - Size(540px, 540px)
 

In [4]:

# overlay_images('Artes/Volpi_Comanda.jpg', 'qrcode.png', position=(320,787), qr_code_size=(540,540))


# Cantina Volpi Mesa

- Background Info:
    - Height: 1772px
    - Widht:  1181px

- Qr Code Info:
    - Position(320, 885)
    - Size(540px, 540px)
- Table Number
    - Position (529, 710)
    - Font: Poppins-Bold.ttf, size 90

In [5]:
font = ImageFont.truetype("Fonts/Poppins-Bold.ttf", 90)
table_number_position = (529, 710)
def table_number(qr_code_image, qr_code_position, qr_code_size, table_number_position, number, color, font): 
    """
    Add a number to a table on an image, centered within a predefined box.

    Parameters:
    - qr_code_image: Image of the QR code
    - qr_code_position: Position of the QR code on the image
    - qr_code_size: Size of the QR code
    - table_number_position: Position for the table number on the image
    - number: Number to be added to the table
    - color: Color of the text
    - font: Font style for the text

    Returns:
    - image_copy: Image with the table number added
    """
    
    background = overlay_images("Artes/Volpi_mesa.jpg", qr_code_image, qr_code_position, qr_code_size=qr_code_size)
    reference_text = "115"
    image_copy = background.copy()
    draw = ImageDraw.Draw(image_copy)
    
    # Calculate the bounding box of the reference text
    reference_bbox = draw.textbbox(table_number_position, reference_text, font=font)
    box_width = reference_bbox[2] - reference_bbox[0]
    box_height = reference_bbox[3] - reference_bbox[1]

    # Calculate the bounding box of the actual text
    text_bbox = draw.textbbox((0, 0), number, font=font)
    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]

    # Calculate the position to center the text within the predefined box
    text_x = table_number_position[0] + (box_width - text_width) / 2
    text_y = table_number_position[1] + (box_height - text_height) / 2

    # Draw the text
    draw.text((text_x, text_y), number, font=font, fill=color)
    
    return image_copy

In [6]:

front_url = "http://localhost:3000"

def generate_qrcode(token, background_color, qrcode_color, transparent):

    QRcode = qrcode.QRCode(error_correction=qrcode.constants.ERROR_CORRECT_H)

    # taking url or text
    url = f"{front_url}/{token}"

    # addingg URL or text to QRcode
    QRcode.add_data(url)

    # generating QR code
    QRcode.make()

    if transparent:
        QRimg = QRcode.make_image(
            fill_color=qrcode_color, back_color="transparent"
        ).convert("RGBA")
        # Remove the transparent margin
        QRimg = QRimg.crop(QRimg.getbbox())
    else:
        QRimg = QRcode.make_image(fill_color=qrcode_color, back_color=background_color).convert("RGB")

    with tempfile.TemporaryDirectory() as tempdir:
        save_path = Path(tempdir) / "image.png"
        QRimg.save(save_path, format="png", dpi=(1000, 1000))

        with open(save_path, "rb") as f:
            bin_data = f.read()

        return base64.b64encode(bin_data).decode("ASCII")

In [13]:
def create_table_qrcodes_arts(data):
    """
    Generate QR codes with table numbers overlayed and save them as images. The data should have both 'qr_code_start' and 'qr_code_end' keys or 'specific_table_number'.

    Parameters:
    - data (dict): A dictionary containing the following keys:
        - 'qr_code_start' (int): The starting number for QR code generation. (Optional)
        - 'qr_code_end' (int): The ending number for QR code generation. (Optional)
        - 'specific_table_number' (list): A list of specific table numbers to generate QR codes for. (Optional)
        - 'token' (str): The token to be used in generating table tokens.
        - 'background_color' (str): The background color of the QR code.
        - 'qrcode_color' (str): The color of the QR code.
        - 'qr_code_position' (tuple): The position of the QR code.
        - 'qr_code_size' (int): The size of the QR code.
        - 'table_number_position' (tuple): The position of the table number overlay.
        - 'background_transparent' (bool): Boolean indicating if the background should be transparent.
        - 'font_path' (str): The path to the font file.

    Returns:
    - None
    """
    
    qr_code_start = data.get('qr_code_start')
    qr_code_end = data.get('qr_code_end')
    specific_table_number = data.get('specific_table_number') 
    token = data.get('token')
    qr_code_color = data.get('qrcode_color')
    qr_code_position = data.get('qr_code_position')
    qr_code_size = data.get('qr_code_size')
    table_number_position = data.get('table_number_position')
    font_path = data.get('font_path')
    comanda = data.get('comanda', False)
    
    if ((not qr_code_start or not qr_code_end and comanda) and not specific_table_number) and comanda == False:
        return ValueError("Either 'qr_code_start' and 'qr_code_end' or 'specific_table_number' must be provided.")
    if not token or not qr_code_color or not qr_code_position or not qr_code_size or not table_number_position or not font_path:
        return ValueError("All 'token', 'background_color', 'qrcode_color', 'qr_code_position', 'qr_code_size', 'table_number_position', and 'font_path' must be provided.")
    if qr_code_start > qr_code_end:
        return ValueError("'qr_code_start' must be less than or equal to 'qr_code_end'.")
    
    font = ImageFont.truetype(font_path, config['font_size'])
    table_qrcodes = []
    
    if comanda == True:
        qr_code = generate_qrcode(token, background_color = "white", qrcode_color= qr_code_color, transparent=True)
        return overlay_images("Artes/Volpi_Comanda.jpg", qr_code, qr_code_position, qr_code_size=qr_code_size)
              
    if not specific_table_number:
        # Gerar os qr Codes
        for i in range(qr_code_start, qr_code_end + 1):
            table_token = f"{token}-{i:04d}"
            table_qrcodes.append((i, generate_qrcode(table_token, background_color= "white", qrcode_color = qr_code_color, transparent=True)))
    else:
        for i in specific_table_number:
            table_token = f"{token}-{i:04d}"
            table_qrcodes.append((i, generate_qrcode(table_token, background_color = "white", qrcode_color= qr_code_color, transparent=True)))
    
    # buffer = io.BytesIO()
    # with ZipFile(buffer, "w") as my_zip:
    #     for number, table_qrcode in table_qrcodes:
    #         img = table_number(table_qrcode, qr_code_position, qr_code_size, table_number_position, str(number), "white", font)
    #         img_bytes = io.BytesIO()
    #         img.save(img_bytes, format='PNG')
    #         img_bytes.seek(0)
    #         my_zip.writestr(f"qrcode_{number:04}.png", img_bytes.read())

    # # Save the ZIP file to disk
    # with open('artes_feitas.zip', 'wb') as f:
    #     f.write(buffer.getvalue())
    # Gerar artes
    for number, table_qrcode in table_qrcodes:
        table_number(table_qrcode, qr_code_position, qr_code_size, table_number_position, str(number), "white", font).save(f'artes_feitas/QrCode_{number}.png')


        

config = {
    "qr_code_start": 1,
    "qr_code_end": 5,
    "specific_table_number": None,
    "token": "wow",
    "qrcode_color": "white",
    "qr_code_position": (320, 885),
    "qr_code_size": (540, 540),
    "table_number_position": (529, 710),
    "font_path": "Fonts/Poppins-Bold.ttf",  # Path to your font file
    "font_size": 90,
    "comanda" : False
}  
 
create_table_qrcodes_arts(config)

In [10]:
import math
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

def insertGreatestCommonDivisors(head):
    current = head

    while current:
        next_node = current.next
        if not next_node:
            break
        middle_node = ListNode(math.gcd(current.val, next_node.val), next_node)
        current.next = middle_node
        current = next_node
    return head

insertGreatestCommonDivisors(ListNode(18, ListNode(6, ListNode(10, ListNode(3,None)))))

In [26]:
import random
class Solution:
    def __init__(self, radius: float, x_center: float, y_center: float):
        self.radius = radius
        self.centerPoint = (x_center, y_center)

    def randPoint(self):
        maxPointX = self.centerPoint[0] + self.radius
        mixPointX = self.centerPoint[0] - self.radius
        maxPointY = self.centerPoint[1] + self.radius
        mixPointY = self.centerPoint[1] - self.radius

        result = []
        result.append(random.uniform(mixPointX,maxPointX))
        result.append(random.uniform(mixPointY,maxPointY))
        return result
obj = Solution(10, 0, 0)
obj.randPoint()

[-3.9567053593403596, 1.2898916831617218]